In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
from PyPDF2 import PdfReader
import gradio as gr
#Gradio is a Python Library that helps us quickly create interactive User Interfaces (UIs) without needing to build a
#frontend from scratch.

In [2]:
load_dotenv(override=True)
client=OpenAI()

In [5]:
reader=PdfReader("me/Neeraj Resume.pdf")
resume=""
for page in reader.pages:
    text= page.extract_text()
    if text:
        resume += text


In [6]:
print(resume)

Neeraj Sandhu
+91 8091220037 |sandhuneeraj.works@gmail.com |LinkedIn: neerajsandhu
Objective
Aspiring frontend developer proficient in HTML, CSS, and JavaScript, passionate about transforming designs
into responsive, user-friendly interfaces. Excited to apply my technical skills and creative problem-solving to
build engaging web experiences in a collaborative environment
Education
Thapar Institute of Engineering and Technology Patiala, Punjab
B.E. Computer Science Engineering 2021 – 2025
St. D. R. Public Sr. Sec. School Una, Himachal Pradesh
Class 12th 2019 – 2020
Parmar International School Una, Himachal Pradesh
Class 10th 2017 – 2018
Projects
VividVue Living |HTML, CSS, JavaScript, Selenium, TestNg
•Developed a user-friendly real estate website with dynamic property listings, advanced search filters, and an
intuitive interface. Used Selenium and TestNG for automation testing based on the design of test cases to ensure
the reliability and proper functionality of the website.
Ecommerce

In [8]:
with open("me/summary.txt", "r", encoding='utf-8') as f:
    summary=f.read()

In [9]:
name = "Neeraj Sandhu"
system_prompt= f"You are acting as {name}. You are answering questions on {name}'s Website, \
    particularly questions related to {name}'s carrer, background, skills and experience. \
    Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
    Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
    If you don't know answer , say so."

system_prompt+=f"\n\n ## Summary:\n{summary}\n\n##{resume}\n\n"
system_prompt+=f"with this context, please chat with the user, always staying in character as {name}"

In [16]:
def chat(message,history):
    messages=[{'role': 'user', 'content': system_prompt}] + history + [{'role': 'user', 'content': message}]
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    return response.choices[0].message.content

In [17]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [ ]:
#Pydantic is used to define, validate, and parse data models in Python, making sure your data is clean , structured and type-safe.
#BaseModel is a core class which is used to create our own data models/schemas.
#used of pydantic :
#1. automatically checks if input data is of the right type.
#2. converts data into correct Python types.
#3. tells us exactly what's wrong if data doesn't match the schema.
#4. can read environment variables/config files and validate them automatically.
from pydantic import BaseModel

#define a schema 
class Evaluation (BaseModel):
    is_acceptable: bool
    feedback: str

In [20]:
evaluator_system_prompt= f"You are an evaluator that decides where a response to a question is acceptable. \
    You are provided with a conversation between a user and an agent. your task is to decide whether the agent's latest response is acceptable or not. \
    The agent is playing the role of {name} and is representing {name} on their website. \
    The agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who come across the website. \
    The agent has been provided with context on {name} in the form of their summary and Resume details. Here is the Information: "

evaluator_system_prompt+=f"\n\n ##Summary: \n{summary}\n\n##Resume details:\n{resume}\n\n"
evaluator_system_prompt+=f"with this context, please evaluate the latest response, replying with whether the response is acceptable or not then recommend the improved answer."

In [ ]:
import os
gemini=OpenAI(
    api_key=os.getenv('G')
)